# Sobre este archivo

La primera entrega pretende ser principalmente exploratoria,
quizás nada del código (si es que hay código) que aparezca 
aquí llegue a la versión final.

La idea es investigar como se podrían hacer las 
funciones principales.
Conviene entonces intentar separa el proyecto.
Una esquema preliminar es el siguiente.

## ¿Qué hay que hacer?

Usar algoritmos genéticos para aproximar una imagen usando figuras.
Inicialmente estas figuras son triángulos, quizás esto cambie.

Se pueden ver los siguientes pasos necesarios (la librería `cv` 
es `cv2` de `numpy`, ver sección siguiente):

1. Leer la imagen del disco y cargarla en python. Función: `cv.imread`
1. Crear una lista inicial de triángulos, de forma aleatoria
    1. Los triángulos deben estar rellenos de color.
       Se deberían tomar los colores de la imagen original
       y usar solo esos.
         1. Obtener colores de la imagen original.
            Hacer una lista con los colores de cada pixel,
            luego usar `numpy.unique` para quitarme
            los repetidos. [Source](https://stackoverflow.com/questions/51705187/list-of-all-colors-in-an-image-using-opencv-and-python)
         1. Dibujar triángulos. Función: `cv.polylines`
    1. Los triángulos deberían llenar un rectángulo del tamaño
       de las dimensiones de la imagen.
1. Dibujar la lista de triángulos en una imagen de las mismas
   dimensiones que la imagen original.
1. Comparar que tan diferentes son las imágenes 
   (pixel a pixel?)
   1. Acceder al valor de pixel de las imágenes con 
      `img.item` 
1. Pasar a la siguiente generación, y volver a dibujar y          comparar; así sucesivamente hasta que la diferencia este
   por debajo de un valor R.
1. Ir guardando que tanto difieren y hacer un grafico al final.
   Esta diferencia debería de converger a cero.

# Librerías de python útiles

+ [Open source computer vision](https://docs.opencv.org/master/index.html). Con esto se puede:
   - Leer una imagen en python del disco
   - Dibujar polígonos e incluso círculos y elipses. Links:
      + (https://docs.opencv.org/master/dc/da5/tutorial_py_drawing_functions.html)
   - Colorear los polígonos dando una tripleta RGB
+ [Numpy](). Con esto se puede (además de `cv2`):
   - Graficar si el programa converge a la imagen o no

# Pruebas
Por ahora las pruebas dependerán bastante de la imágen específica de *Freedo*
que estoy usando, es decir, las dimensiones de la imágen por ejemplo
tendrían que leerse de forma dinámica de la imágen suministrada.

Como esto hay muchas otras cosas que cambiarán antes de la versión final.

No estan implementadas aun las funciones como tal,
el codigo que existe si puede adaptarse a definiciones de funciones mas adelante.

## Leer la imágen de un url o archivo local y mostrarla

In [ ]:
# https://stackoverflow.com/questions/21061814/how-can-i-read-an-image-from-an-internet-url-in-python-cv2-scikit-image-and-mah
from skimage import io
import cv2
# Dimensions 209x204
img = io.imread("./Freedo_improved.jpeg")
#cv2.imshow("window",img)
#cv2.waitKey(0)

## Leer valores de la imagen a evolucionar
la idea es tomar los colores de la imagen a evolucionar, esto aún no funciona
correctamente.

In [ ]:
# https://stackoverflow.com/questions/51705187/list-of-all-colors-in-an-image-using-opencv-and-python
from skimage import io
import numpy as np
import cv2
# Dimensions 209x204
img = io.imread("./Freedo_improved.jpeg")
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

all_rgb_codes = img.reshape(-1, img.shape[-1])
unique_rgbs = np.unique(all_rgb_codes, axis=0)

print(unique_rgbs)

## Unas imagenes a partir de una lista de poligonos
Los polígonos pueden tener cualquier color dado en una tripleta RGB.
Vale la pena transparencia?

Este puede servir como el primer ADN, donde los individuos se generan al azar.
Tambien es la primera población, pues la lista imagearr es una lista de imagenes
con triangulos dibujados.

In [ ]:
# https://docs.opencv.org/master/dc/da5/tutorial_py_drawing_functions.html
import numpy as np
import random
import cv2

N = 15 # N triangles
P = 25 # Population
X,Y = 204,209 # picture dimensions

imagearr = []
for i in range (0,P):
    image = np.ones((X,Y,3), np.uint8)*255 # Blank squared image
    imagearr = imagearr + [image] # Fill the list with P blank images

pts = []
# This loop randomly generates the triangles and draws them
for k in range (0,P):
    for i in range(0,N):
        pts =  np.array(
                [[random.randint(0,X),random.randint(0,Y)],
                 [random.randint(0,X),random.randint(0,Y)],
                 [random.randint(0,X),random.randint(0,Y)]],
                np.int32
        )
        pts = pts.reshape((-1,1,2))
        cv2.fillPoly(
            imagearr[k], # Draw to the k blank image
            [pts],
            (random.randint(0,255),random.randint(0,255),random.randint(0,255)) # Random colors for triangles
        )

#cv2.imshow("window2",imagearr[15])
#cv2.waitKey(0)
#cv2.destroyWindow('window2')
#cv2.waitKey(1)

## Comparar imágenes
La idea es utilizar la función `cv2.subtract` para restar las imágenes.
Pongamos el resultado de la resta en una variable `diff`.
Luego, se puede usar `np.sum` para sumar los elementos del array `diff`.
Si esta suma es cero, quiere decir que las imágenes son iguales.
Por lo que buscamos que el resultado de esta suma sea la menor posible.

In [ ]:
# https://medium.com/@jineshjohn/image-compare-using-opencv-python-ac9560be2d96
difflist = []
for i in range(0,P):
    diff = np.sum(cv2.subtract(img, imagearr[i]))
    difflist += [diff]
min(difflist)

La variable `diff` puede irse guardando para luego graficarla y ver si
el algoritmo esta funcionando o no.
`min(difflist)` da el más parecido de esta generación, lo cual seguramente sea útil.

## Cruce y mutaciones
Para hacer el cruce de dos individuos podemos tomar mitad de los triangulos de uno
y mitad de los triangulos del otro, y producir un nuevo individuo. La variable `mutated` indicará, una vez la función este bien construida, si hay que hacer alguna mutación al hijo.

Los hijos deben heredar parte de los triángulos y los padre y, además, el color de dichos triángulos.

In [ ]:
import random

X,Y = 204,209

N = 10

indv1 = []
indv2 = []

for i in range (0,N): # Individuals with 10 triangles
    indv1 += [
        [
            [random.randint(0,X),random.randint(0,Y)],
            [random.randint(0,X),random.randint(0,Y)],
            [random.randint(0,X),random.randint(0,Y)],
            [random.randint(0,255),random.randint(0,255),random.randint(0,255)], # Color
        ]
    ]
    indv2 += [
        [
            [random.randint(0,X),random.randint(0,Y)],
            [random.randint(0,X),random.randint(0,Y)],
            [random.randint(0,X),random.randint(0,Y)],
            [random.randint(0,255),random.randint(0,255),random.randint(0,255)], # Color
        ]
    ]
    
print(indv1)
print()
print(indv2)
print()

son = []
for i in range (0,N):
    if i < N//2:
        son += [indv1[i]]
    else:
        son += [indv2[i]]

print(son)

mutated = False
if random.randint(0,100) == 1: # event with 1% chance of ocurring
    mutated = True

print(mutated)
# Returns: son,mutated

Para mutar un individuo se pueden alterar sus atributos ligeramente.
En el caso de los triángulos que tenemos, los atributos son:

1. La posición de los vértices de los triángulos y,
1. el color de los triángulos.

Las mutaciones deberían tener poca probabilidad de ocurrir.
Las mutaciones deberían ocurrir despues de un cruce.
Entonces la función de cruce debe devolver un valor booleano
que indique si hay que hacer una mutación, o no.

El código de abajo esta incompleto.
La idea es:

1. Calcular la cantidad de triangulos que se van a mutar y tambien
   cuantos atributos de dichos triangulos se van a cambiar.

In [ ]:
import random

N = 10 # Same as last code snippet
mutated = True # force mutation

if mutated == True:
    M = random.randint(0,N) # Amount of triangles to be modified
    K = random.randint(0,4) # Amount of attributtes to be modified
    for i in range (0,M):
        randtriag = son[random.randint(0,N)] # Take a random triangle from son
        for i in range (0,K):
            k = random.randint(0,4) # pick random attributte
            if k != 3: # if a vertex got picked
                randtriag[k] = randtriag[k]
            else: # if color got picked

## Fitness